In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from db_initialize import *
# here we assume that database.db only contains data from healingwell
engine = create_engine('sqlite://///Users/Johnson/Documents/nit/thesis/forum_crawler/database.db')
Session = sessionmaker(bind=engine)
Base.metadata.bind = engine
session = Session()

In [35]:
thread = session.query(Thread).get(20415)
thread


Title: Small Volume Cancer...PSA 20 Gleason 8....anyone similar 
User: soccerj3 
Time: 2016-09-02 20:31:00 
Posts: 17
URL: http://www.healingwell.com/community/default.aspx?f=35&m=587640 
Hi, 

In [36]:
thread.posts

[
 User: spinbiscuit 
 Time: 2006-09-04 06:50:00 
 Thread: Small Volume Cancer...PSA 20 Gleason 8....anyone similar 
 Order: None 
 URL: http://www.healingwell.com/community/default.aspx?f=35&m=587640 
 Hello soccerj3,
 The high readings of the Gleason and PSA are of concern, but the urologist said SMALL VOLUME with only one of eleven cores possive for PCa that is HOPEFUL. In any event the bone and cat scans will give a better indication of what your dad is facing.
 He has to take heart in knowing that what ever the tests show there will be a state of the art therapy availabe to treat his PCa. The advance in the treatment technologies over the past 5-10 years has made PCa the most curable of all the cancers. First thing is know what you're dealing with; get control of the emotions, and then you can form an effective strategy to beat this PCa.
 You have found the right sight. Everyone here will help you and your dad get through this most difficult time. We are all going through it as we

In [25]:
from datetime import datetime
def get_time_span(thread):
    #post_timestamps = np.array([post.timestamp for post in thread.posts], dtype='datetime64')
    # assume the last post in thread.posts is the last reply
    thread_time = datetime.strptime(thread.timestamp, '%Y-%m-%d %H:%M:%S')
    if not thread.posts:
    	return 0
    last_reply_time = datetime.strptime(thread.posts[-1].timestamp, '%Y-%m-%d %H:%M:%S')
    print(thread_time)
    print(last_reply_time)
    time_delta = last_reply_time - thread_time
    print(time_delta)
    #print(time_delta)
    #calculate the time span in hours
    time_span = 24*int(time_delta.days) + time_delta.seconds//3600
    if time_span < 0:
    	print('Found unusual thread, ID is %d' % thread.id)
    return time_span

In [26]:
get_time_span(thread)

2016-09-02 08:19:00
2016-06-15 10:49:00
-79 days, 2:30:00
Found unusual thread, ID is 505


-1894

In [32]:
from datetime import datetime
def time_parser(body):
    parts = body.split()
    if parts[1] == 'Today':
        today = datetime.today()
        date = [today.month, today.day, today.year]
    elif parts[1] == 'Yesterday':
        today = datetime.today() - timedelta(1)
        date = [today.month, today.day, today.year]
    else:
        date = parts[1].split('/')
        date = [int(x) for x in date]
    time = parts[2].split(':')
    time = [int(x) for x in time]
    #convert AM/PM to 24 hour
    if parts[3]=='PM' and 1<=time[0]<=11:
        time[0] += 12
    elif parts[3]=='AM' and time[0]==12:
        time[0] -= 12
    else:
        pass
    dtime = datetime(date[2], date[0], date[1], time[0], time[1])
    return dtime.isoformat(sep=' ')

In [33]:
time_parser("\xa0\xa0\xa0Posted 9/3/2015 3:22 PM (GMT -6)\xa0\xa0\xa0")

'2015-09-03 15:22:00'

In [48]:
post = "<div class=\"PostBoxWrapper\">                <div class=\"PostMessageBody\">Hi Soccerj3 &amp; Dad,<br><br>Keep in touch... we are all here to help you both get through this!!!  <br><br>\"GOD Bless You\" <br>It's a little prayer \"GOD Bless You\"... but it means so much each day,<br>It means may angels guard you and guide you on your way.  ~author unknown~<br><br>Take care and know we care.<br>Lee &amp; Buddy<hr class=\"PostHR\"><font color=\"blue\"><i>mama bluebird</i></font></div>\
<div><i><font color=\"orange\">~&nbsp; ~&nbsp; ~&nbsp; ~&nbsp; ~&nbsp;&nbsp;~&nbsp; ~&nbsp;&nbsp;~&nbsp; ~&nbsp; ~&nbsp; ~&nbsp; ~&nbsp; ~</font></i></div>\
<div><i><font color=\"#ffa500\">Buddy 53 RRP April 3, 2006 PSA 4.6 Gleason 3+3=6 T2a Confined to Prostate </font></i></div>\
<div><i><font color=\"#ffa500\">June 29th PSA Less than 0.1 Non-detectable&nbsp; </font></i><p></p></div>                <div class=\"PostToTopLink\"><a href=\"#ttop\">Back to Top</a></div>            </div>"

In [49]:
from bs4 import BeautifulSoup
def body_parser(body):
    if body.find('<div class="PostToTopLink">') != -1:
        body = body[:body.find('<div class="PostToTopLink">')]
    if body.find('<hr class="PostHR">') != -1:
        body = body[:body.find('<hr class="PostHR">')]
    body = body.replace("<br>","NEWLINE")
    soup = BeautifulSoup(body, 'html.parser')
    text = soup.get_text()
    return text.replace("NEWLINE", "\n")  

In [50]:
print(body_parser(post))

 Hi Soccerj3 & Dad,

Keep in touch... we are all here to help you both get through this!!!  

"GOD Bless You" 
It's a little prayer "GOD Bless You"... but it means so much each day,
It means may angels guard you and guide you on your way.  ~author unknown~

Take care and know we care.
Lee & Buddy


In [ ]:

with open("/debug/wrong_thread_time.txt", "w") as text_file:
    text_file.write("Purchase Amount: %s" % TotalAmount)

In [73]:
def check_time_span(post, thread):
    thread_time = datetime.strptime(thread.timestamp, '%Y-%m-%d %H:%M:%S')
    reply_time = datetime.strptime(post.timestamp, '%Y-%m-%d %H:%M:%S')
    time_delta = reply_time - thread_time
    #print(time_delta)
    #calculate the time span in hours
    time_span = 24*int(time_delta.days) + time_delta.seconds//3600
    if time_span < 0:
        req = requests.get(url=post.url)
        with open(("./wrong_time_span/%d.html" % post.thread_id), "w") as text_file:
            print(req.text)
            text_file.write(req.text)

In [74]:
post = thread.posts[0]
save_wrong_time_thread(post)


<!doctype html>
<html class="no-js" lang="en">

  <head>
  	
<title>Small Volume Cancer...PSA 20 Gleason 8....anyone similar - Prostate Cancer - HealingWell.com Forum</title>
<meta name="robots" content="index,follow" />
    
<link rel="canonical" href="http://www.healingwell.com/community/default.aspx?f=35&m=587640" />        
    <meta http-equiv="X-UA-Compatible" content="IE=edge" /> 
    <meta charset="utf-8" />
    <link href="../generic_community.css" type="text/css" rel="stylesheet" />
    <link rel="stylesheet" type="text/css" href="/community/styles/HealingWell/css/style.css" />
    <script src="/community/js/TBMain.js" type="text/javascript"></script>
<script type="text/javascript">
	function launchChat(){ 
		window.open ("/community/chat/default.asp","chatWindow","height=625,width=740,top=130,left=150,scrollbars=yes,resizable=yes,status=0");
	}
</script>

    
        <!-- Begin EHS Head Tag -->
        <script type="text/javascript">
			//<![CDATA[
			// Please remove % si

In [ ]:
thread_time = datetime.strptime(thread.timestamp, '%Y-%m-%d %H:%M:%S')
last_reply_time = datetime.strptime(thread.posts[-1].timestamp, '%Y-%m-%d %H:%M:%S')
time_delta = last_reply_time - thread_time
#print(time_delta)
#calculate the time span in hours
time_span = 24*int(time_delta.days) + time_delta.seconds//3600
